In [1]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime, timedelta
from pandas import DataFrame, concat, date_range, ExcelWriter, to_datetime
import os
from numpy import isnan, inf, mean
import time
import json
from Calculator import Calculator as Calc

from utils import getSchema, changedType

parent = os.path.dirname(os.path.abspath("__file__"))
output_path = os.path.join(parent, "Output", "GoldCross")
if not os.path.isdir(output_path):
    os.makedirs(output_path)

c:\users\kevin\appdata\local\programs\python\python37\lib\site-packages\mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


In [2]:
def StockInterDay(start_date:[str,datetime]=datetime(2021,4,14), 
                  end_date:[str,datetime]=datetime(2021,4,14),
                  tickers:[str,list]=''):
    if isinstance(start_date, datetime):
        start_date = start_date.strftime("%Y-%m-%d")
    if isinstance(end_date, datetime):
        end_date = end_date.strftime("%Y-%m-%d")
    if isinstance(tickers, str):
        tickers = tickers.split(',')
    schema = getSchema('TWSE')
    table = schema['historicalPrice']
    data = list(table.find({'Date':{'$gte':start_date, '$lte':end_date}, "Ticker" :{"$in":tickers}}))
    return data

def StockList():
    schema = getSchema('TWSE')
    table = schema['StockList']
    last_date = sorted(table.distinct("UpdateDate"))[-1]
    data = list(table.find({"UpdateDate":{"$eq":last_date}, "Industry" :{"$ne":""}}))
    return data

In [3]:
def get_commission(price:float, multiplier:int=1000, qty=1, long:bool=1, dayTrade:bool=False):
    """
    計算個別部位的單邊交易成本

    Params:
        symbol : 商品代碼
        exchange : 交易所
        cost : 交易價格
        multiplier : 價格還原現金之乘數
            例如:
                股票 : 1張 = 1,000股，10元的股票還原現金價值，即為10 *1,000 = 10,000元
                期貨 : 台指期1點200元，假設現在10,000點，則一口台股的價值為 200 * 10,000 = 2,000,000
        qty : 買賣口數或張數
        Real : 是否為實單, default = False
        direction : 交易方向 進場(買賣)或出場
            P.S. 股票交易的交易稅是出場才計算
    """
    commission = price * (0.1425 / 100) * multiplier * qty
    commission = 20 if commission < 20 else commission
    tax = price * (0.3 / 100) * multiplier * qty
    if dayTrade:
        fee /= 2
#     tradeCost = commission# * 0.6
    if not long:
        return commission, tax
    return commission, 0

In [4]:
def CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos:int = 1, qty:int = 1, multiplier:int=1000):
    pnl = (exit_price - entry_price) * pos * qty * multiplier
    entry_com, entry_tax = get_commission(entry_price, multiplier, qty, long = pos > 0)
    exit_com, exit_tax = get_commission(exit_price, multiplier, qty, long = not (pos > 0))
#     print(entry_date, exit_date, entry_price, exit_price,entry_com,entry_tax,exit_com,exit_tax)
    return {
        'EntryDate':entry_date,
        'ExitDate':exit_date,
        'EntryPrice':entry_price,
        'ExitPrice':exit_price,
        'EntryCommission':round(entry_com),
        'ExitCommission':round(exit_com),
        'EntryTax':round(entry_tax),
        'ExitTax':round(exit_tax),
        'TotalCost':round(entry_com)+round(exit_com)+round(entry_tax)+round(exit_tax),
        'HoldingPeriod':(exit_date-entry_date).days,
        'Net':round(pnl - (round(entry_com)+round(exit_com)+round(entry_tax)+round(exit_tax))),
        'Ret':round((pnl - (round(entry_com)+round(exit_com)+round(entry_tax)+round(exit_tax))) / (entry_price * 1000),4),
        'MaxPriceBetweeenHolding':max_price,
        'MaxRetBetweeenHolding':round(max_price/entry_price-1,4),
        'MinPriceBetweeenHolding':min_price,
        'MinRetBetweeenHolding':round(min_price/entry_price-1,4)
    }

In [5]:
def GoldCross(df:DataFrame, **kwargs):
    entry_date = None
    exit_date = None
    entry_price = 0
    exit_price = 0
    max_price = 0
    min_price = inf
    sig = pos = 0
#     num_breakout_day = int(kwargs.get('num_breakout_day', 100))
    take_profit = float(kwargs.get('take_profit', .1))
    stop_loss = float(kwargs.get('stop_loss', .1))
    
    
    df['MA20'] = Calc.MA(df.fillna(method='ffill'), [20]).MA20
    df['EMA12'] = Calc.EMA(df.fillna(method='ffill'), [12]).EMA12
    result = []
    for i, row in enumerate(df.itertuples()):
        if not i: continue
        # Check Signal without pos
        if not sig and not pos:
            if row.EMA12 > row.MA20 and df.EMA12.iloc[i-1] <= df.MA20.iloc[i-1]:
                sig = 1
        # Entry Market
        elif sig and not pos:
#             if row.Ticker == '1538':print(row)
            if not row.Volume or isnan(row.Open): continue
#             print(row)
            pos, sig = sig, 0
            max_price = min_price = entry_price = row.Open
            entry_date = row.Date
            max_price = max(row.High, max_price)
            min_price = min(row.Low, min_price)
        # Check Signal with pos
        elif not sig and pos:
            max_price = max(row.High, entry_price)
            min_price = min(row.Low, entry_price)
            if (row.High / entry_price - 1 >= take_profit):
                sig = -1
#                 exit_price = row.High
            if (row.Low / entry_price - 1 <= -stop_loss):
                sig = -1
#                 exit_price = row.Low
        # Exit Market
        elif sig and pos:
            
            if not row.Volume or isnan(row.Open): continue
#             if row.Ticker == '1538':print(row)
            exit_price = row.Open
            exit_date = row.Date
            res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
            result.append(res)
            entry_date = None
            exit_date = None
            entry_price = 0
            exit_price = 0
            max_price = 0
            min_price = inf
            sig = pos = 0
        if i == df.shape[0]-1 and pos:
            exit_price = row.Close
            if not row.Volume or isnan(row.Open):
                exit_price = df.fillna(method='ffill').Close.iloc[-1]
            
            exit_date = row.Date
            res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
            result.append(res)
            entry_date = None
            exit_date = None
            entry_price = 0
            exit_price = 0
            max_price = 0
            min_price = inf
            sig = pos = 0
#     print()
            
    return result

In [6]:
def Backtest(strategy:callable, ticker:str, dt:datetime=datetime.today(), bt_period = 5, params:dict={}):
    data = StockInterDay(dt+timedelta(-365*bt_period), dt, ticker)
    if not data:return []
    df = DataFrame(data)
    print(f"From {df.Date.iloc[0]} to {df.Date.iloc[-1]}")
#     print(df.iloc[0])
    df.Date = to_datetime(df.Date)
    for col in 'Open,High,Low,Close,Volume'.split(','):
        df[col] = df[col].apply(changedType)
    return strategy(df, **params)
    

In [7]:
bt_list = StockList()

In [8]:
results = {}
for ticker_info in bt_list:
    print(f"============= Backtest {ticker_info['Ticker']}=============")
    results[ticker_info['Ticker']] = Backtest(GoldCross, ticker_info['Ticker'], bt_period = 10)
#     break

============= Backtest 1101=============
From 2013-08-05 to 2023-08-02
============= Backtest 1102=============
From 2013-08-05 to 2023-08-02
============= Backtest 1103=============
From 2013-08-05 to 2023-08-02
============= Backtest 1104=============
From 2013-08-05 to 2023-08-02
============= Backtest 1108=============
From 2013-08-05 to 2023-08-02
============= Backtest 1109=============
From 2013-08-05 to 2023-08-02
============= Backtest 1110=============
From 2013-08-05 to 2023-08-02
============= Backtest 1201=============
From 2013-08-05 to 2023-08-02
============= Backtest 1203=============
From 2013-08-05 to 2023-08-02
============= Backtest 1210=============
From 2013-08-05 to 2023-08-02
============= Backtest 1213=============
From 2013-08-05 to 2023-08-02
============= Backtest 1215=============
From 2013-08-05 to 2023-08-02
============= Backtest 1216=============
From 2013-08-05 to 2023-08-02
============= Backtest 1217=============
From 2013-08-05 to 2023-08-02
======

From 2013-08-05 to 2023-08-02
============= Backtest 1527=============
From 2013-08-05 to 2023-08-02
============= Backtest 1528=============
From 2013-08-05 to 2023-08-02
============= Backtest 1529=============
From 2013-08-05 to 2023-08-02
============= Backtest 1530=============
From 2013-08-05 to 2023-08-02
============= Backtest 1531=============
From 2013-08-05 to 2023-08-02
============= Backtest 1532=============
From 2013-08-05 to 2023-08-02
============= Backtest 1533=============
From 2013-08-05 to 2023-08-02
============= Backtest 1535=============
From 2013-08-05 to 2023-08-02
============= Backtest 1536=============
From 2013-08-05 to 2023-08-02
============= Backtest 1537=============
From 2013-08-05 to 2023-08-02
============= Backtest 1538=============
From 2013-08-05 to 2023-08-02
============= Backtest 1539=============
From 2013-08-05 to 2023-08-02
============= Backtest 1540=============
From 2013-08-05 to 2023-08-02
============= Backtest 1541=============
From 2

From 2013-08-05 to 2023-08-02
============= Backtest 2103=============
From 2013-08-05 to 2023-08-02
============= Backtest 2104=============
From 2013-08-05 to 2023-08-02
============= Backtest 2105=============
From 2013-08-05 to 2023-08-02
============= Backtest 2106=============
From 2013-08-05 to 2023-08-02
============= Backtest 2107=============
From 2013-08-05 to 2023-08-02
============= Backtest 2108=============
From 2013-08-05 to 2023-08-02
============= Backtest 2109=============
From 2013-08-05 to 2023-08-02
============= Backtest 2114=============
From 2013-08-05 to 2023-08-02
============= Backtest 2115=============
From 2013-12-25 to 2023-08-02
============= Backtest 2201=============
From 2013-08-05 to 2023-08-02
============= Backtest 2204=============
From 2013-08-05 to 2023-08-02
============= Backtest 2206=============
From 2013-08-05 to 2023-08-02
============= Backtest 2207=============
From 2013-08-05 to 2023-08-02
============= Backtest 2208=============
From 2

From 2013-08-05 to 2023-08-02
============= Backtest 2440=============
From 2013-08-05 to 2023-08-02
============= Backtest 2441=============
From 2013-08-05 to 2023-08-02
============= Backtest 2442=============
From 2013-08-05 to 2023-08-02
============= Backtest 2443=============
From 2013-08-05 to 2023-08-02
============= Backtest 2444=============
From 2013-08-05 to 2023-08-02
============= Backtest 2449=============
From 2013-08-05 to 2023-08-02
============= Backtest 2450=============
From 2013-08-05 to 2023-08-02
============= Backtest 2451=============
From 2013-08-05 to 2023-08-02
============= Backtest 2453=============
From 2013-08-05 to 2023-08-02
============= Backtest 2454=============
From 2013-08-05 to 2023-08-02
============= Backtest 2455=============
From 2013-08-05 to 2023-08-02
============= Backtest 2457=============
From 2013-08-05 to 2023-08-02
============= Backtest 2458=============
From 2013-08-05 to 2023-08-02
============= Backtest 2459=============
From 2

From 2013-08-05 to 2023-08-02
============= Backtest 2845=============
From 2013-08-05 to 2023-08-02
============= Backtest 2849=============
From 2013-08-05 to 2023-08-02
============= Backtest 2850=============
From 2013-08-05 to 2023-08-02
============= Backtest 2851=============
From 2013-08-05 to 2023-08-02
============= Backtest 2852=============
From 2013-08-05 to 2023-08-02
============= Backtest 2855=============
From 2013-08-05 to 2023-08-02
============= Backtest 2867=============
From 2013-08-05 to 2023-08-02
============= Backtest 2880=============
From 2013-08-05 to 2023-08-02
============= Backtest 2881=============
From 2013-08-05 to 2023-08-02
============= Backtest 2882=============
From 2013-08-05 to 2023-08-02
============= Backtest 2883=============
From 2013-08-05 to 2023-08-02
============= Backtest 2884=============
From 2013-08-05 to 2023-08-02
============= Backtest 2885=============
From 2013-08-05 to 2023-08-02
============= Backtest 2886=============
From 2

...

From 2013-08-05 to 2023-08-01
============= Backtest 4905=============
From 2013-08-05 to 2023-08-01
============= Backtest 4907=============
From 2013-08-05 to 2023-08-01
============= Backtest 4908=============
From 2013-08-05 to 2023-08-01
============= Backtest 4909=============
From 2013-08-05 to 2023-08-01
============= Backtest 4911=============
From 2013-08-05 to 2023-08-01
============= Backtest 4924=============
From 2013-08-05 to 2023-08-01
============= Backtest 4931=============
From 2013-08-05 to 2023-08-01
============= Backtest 4933=============
From 2013-08-05 to 2023-08-01
============= Backtest 4939=============
From 2013-08-05 to 2023-08-01
============= Backtest 4944=============
From 2013-08-05 to 2023-08-01
============= Backtest 4945=============
From 2020-11-20 to 2023-08-01
============= Backtest 4946=============
From 2013-08-05 to 2023-08-01
============= Backtest 4950=============
From 2013-08-05 to 2023-08-01
============= Backtest 4953=============
From 2

From 2013-08-05 to 2023-08-01
============= Backtest 5530=============
From 2013-08-05 to 2023-08-01
============= Backtest 5536=============
From 2013-08-05 to 2023-08-01
============= Backtest 5543=============
From 2016-05-09 to 2023-08-01
============= Backtest 5601=============
From 2013-08-05 to 2023-08-01
============= Backtest 5603=============
From 2013-08-05 to 2023-08-01
============= Backtest 5604=============
From 2013-08-05 to 2023-08-01
============= Backtest 5609=============
From 2013-08-05 to 2023-08-01
============= Backtest 5701=============
From 2013-08-05 to 2023-08-01
============= Backtest 5703=============
From 2013-08-05 to 2023-08-01
============= Backtest 5704=============
From 2013-08-05 to 2023-08-01
============= Backtest 5864=============
From 2013-08-05 to 2023-08-01
============= Backtest 5878=============
From 2014-10-28 to 2023-08-01
============= Backtest 5902=============
From 2013-08-05 to 2023-08-01
============= Backtest 5903=============
From 2

From 2013-08-05 to 2023-08-01
============= Backtest 6291=============
From 2013-08-05 to 2023-08-01
============= Backtest 6292=============
From 2013-08-05 to 2023-08-01
============= Backtest 6294=============
From 2013-08-05 to 2023-08-01
============= Backtest 6404=============
From 2014-07-24 to 2023-08-01
============= Backtest 6411=============
From 2014-03-11 to 2023-08-01
============= Backtest 6417=============
From 2015-10-08 to 2023-08-01
============= Backtest 6418=============
From 2013-08-05 to 2023-08-01
============= Backtest 6419=============
From 2014-12-29 to 2023-08-01
============= Backtest 6425=============
From 2013-08-07 to 2023-08-01
============= Backtest 6426=============
From 2014-12-17 to 2023-08-02
============= Backtest 6432=============
From 2015-05-20 to 2023-08-01
============= Backtest 6435=============
From 2016-01-07 to 2023-08-01
============= Backtest 6441=============
From 2013-12-02 to 2023-08-01
============= Backtest 6446=============
From 2

From 2013-08-05 to 2023-08-01
============= Backtest 8068=============
From 2013-08-05 to 2023-08-01
============= Backtest 8069=============
From 2013-08-05 to 2023-08-01
============= Backtest 8071=============
From 2013-08-05 to 2023-08-01
============= Backtest 8074=============
From 2013-08-05 to 2023-08-01
============= Backtest 8076=============
From 2013-08-05 to 2023-08-01
============= Backtest 8077=============
From 2013-11-22 to 2023-08-01
============= Backtest 8080=============
From 2013-08-05 to 2023-08-01
============= Backtest 8083=============
From 2013-08-05 to 2023-08-01
============= Backtest 8084=============
From 2013-08-05 to 2023-08-01
============= Backtest 8085=============
From 2013-08-05 to 2023-08-01
============= Backtest 8086=============
From 2013-08-05 to 2023-08-01
============= Backtest 8087=============
From 2013-08-05 to 2023-08-01
============= Backtest 8088=============
From 2013-08-05 to 2023-08-01
============= Backtest 8089=============
From 2

From 2013-08-05 to 2023-08-02
============= Backtest 2881C=============
From 2021-10-29 to 2023-08-02
============= Backtest 6761=============
From 2021-11-08 to 2023-08-01
============= Backtest 6806=============
From 2021-11-15 to 2023-08-02
============= Backtest 2945=============
From 2021-11-30 to 2023-08-02
============= Backtest 6770=============
From 2021-12-06 to 2023-08-02
============= Backtest 6792=============
From 2021-12-08 to 2023-08-02
============= Backtest 4770=============
From 2021-12-22 to 2023-08-02
============= Backtest 2883B=============
From 2021-12-30 to 2023-08-02
============= Backtest 6691=============
From 2022-01-03 to 2023-08-02
============= Backtest 3592=============
From 2013-08-05 to 2023-08-02
============= Backtest 6719=============
From 2022-01-13 to 2023-08-02
============= Backtest 2756=============
From 2022-01-14 to 2023-08-01
============= Backtest 5244=============
From 2013-08-05 to 2023-08-02
============= Backtest 6693=============
From

# Summary

In [9]:
results['1101']

[{'EntryDate': Timestamp('2013-11-20 00:00:00'),
  'ExitDate': Timestamp('2014-02-26 00:00:00'),
  'EntryPrice': 43.2,
  'ExitPrice': 47.15,
  'EntryCommission': 62,
  'ExitCommission': 67,
  'EntryTax': 0,
  'ExitTax': 141,
  'TotalCost': 270,
  'HoldingPeriod': 98,
  'Net': 3680,
  'Ret': 0.0852,
  'MaxPriceBetweeenHolding': 48.45,
  'MaxRetBetweeenHolding': 0.1215,
  'MinPriceBetweeenHolding': 43.2,
  'MinRetBetweeenHolding': 0.0},
 {'EntryDate': Timestamp('2014-04-02 00:00:00'),
  'ExitDate': Timestamp('2014-12-18 00:00:00'),
  'EntryPrice': 47.75,
  'ExitPrice': 41.85,
  'EntryCommission': 68,
  'ExitCommission': 60,
  'EntryTax': 0,
  'ExitTax': 126,
  'TotalCost': 254,
  'HoldingPeriod': 260,
  'Net': -6154,
  'Ret': -0.1289,
  'MaxPriceBetweeenHolding': 47.75,
  'MaxRetBetweeenHolding': 0.0,
  'MinPriceBetweeenHolding': 41.55,
  'MinRetBetweeenHolding': -0.1298},
 {'EntryDate': Timestamp('2015-01-29 00:00:00'),
  'ExitDate': Timestamp('2015-06-05 00:00:00'),
  'EntryPrice': 42.

In [10]:
summary_ = []
bt_day = datetime.today()
bt_day_str = bt_day.strftime("%Y%m%d")
for ticker, result in results.items():
    total_cost = 0
#     total_pnl = 0
    total_net = 0
    trade_num = len(result)
    win_num = 0
    loss_num = 0
    hold_period = []
    rets = []
    for res in result:
        total_net += res['Net']
        total_cost += res['TotalCost']
        win_num += int(res['Net'] > 0)
        loss_num += int(res['Net'] < 0)
        hold_period.append(res['HoldingPeriod'])
        rets.append(res['Ret'])
    summary_.append({
        '代號':ticker,
        '總成本':total_cost,
        '總損益(淨)':total_net,
        '獲利次數':win_num,
        '損失次數':loss_num,
        '總交易次數':trade_num,
        '勝率%':round(win_num / trade_num*100,2) if trade_num else 0,
        '平均報酬':mean(rets),
        '平均持倉時間(日)':mean(hold_period)
    })
    if result:
        DataFrame(result).to_csv(os.path.join(output_path, f'{ticker}_{bt_day_str}.csv'), index=False)
        

In [11]:
sum_df = DataFrame(summary_)#.sort_values(['勝率%','總交易次數'], ascending=False)

In [12]:
sum_df['總損益(淨)'].sum()

4338267

In [13]:
sum_df

,代號,總成本,總損益(淨),獲利次數,損失次數,總交易次數,勝率%,平均報酬,平均持倉時間(日)
0,1101,6409,-14509,13,15,28,46.43,-0.005925,87.857143
1,1102,5005,295,13,10,23,56.52,0.007665,107.000000
2,1103,2866,9214,19,11,30,63.33,0.024170,71.833333
3,1104,2760,3290,11,9,20,55.00,0.014050,130.650000
4,1108,1599,4751,14,8,22,63.64,0.022645,116.727273
...,...,...,...,...,...,...,...,...,...
1813,6657,0,0,0,0,0,0.00,NaN,NaN
1814,6877,265,-6415,0,1,1,0.00,-0.128300,4.000000
1815,2073,0,0,0,0,0,0.00,NaN,NaN
1816,6902,0,0,0,0,0,0.00,NaN,NaN


In [14]:
sum_df.to_csv(os.path.join(output_path, 'Summary.csv'),index=False, encoding='utf-8-sig')

In [15]:
sum_df_prob_sorted = sum_df[sum_df['勝率%'] > 65]
sum_df_trade_num_sorted = sum_df[sum_df.總交易次數 >= 20]
sum_df_holding_period_sorted = sum_df[sum_df['平均持倉時間(日)'] <= 40]

In [16]:
# top_num = 100
prob_Ticker = sum_df_prob_sorted.代號 # .iloc[:top_num]
trade_num_Ticker = sum_df_trade_num_sorted.代號 # .iloc[:top_num]
holding_period_Ticker = sum_df_holding_period_sorted.代號
suitable_Ticker = list(set(prob_Ticker).intersection(trade_num_Ticker).intersection(holding_period_Ticker))

In [17]:
print(sum_df[sum_df.代號.isin(suitable_Ticker)].shape)
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False).head(50)

(53, 9)


,代號,總成本,總損益(淨),獲利次數,損失次數,總交易次數,勝率%,平均報酬,平均持倉時間(日)
306,2379,55510,418190,32,9,41,78.05,0.054239,36.048780
814,6558,5400,26990,19,7,26,73.08,0.043773,23.884615
1552,6548,26078,216972,21,8,29,72.41,0.044497,35.793103
1140,3581,4271,22629,26,10,36,72.22,0.037753,32.750000
1532,6488,102323,485977,30,12,42,71.43,0.041269,23.452381
581,3432,4311,22479,33,14,47,70.21,0.039643,29.425532
869,8374,5479,35741,28,12,40,70.00,0.037707,38.900000
1518,6432,6594,42156,25,11,36,69.44,0.043081,34.388889
779,6243,5925,68005,34,15,49,69.39,0.074214,21.489796
762,6196,17970,86230,31,14,45,68.89,0.039427,33.466667


In [37]:
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False).to_csv(os.path.join(output_path, 'Summary_prob_holding_tradenum.csv'), index=False)

In [36]:
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False).sum()#describe()

代號           6.558655e+155
總成本           6.819490e+05
總損益(淨)        3.469261e+06
獲利次數          1.089000e+03
損失次數          5.170000e+02
總交易次數         1.606000e+03
勝率%           2.657180e+03
平均報酬          1.329695e+00
平均持倉時間(日)     1.179085e+03
dtype: float64